# 2장 서울시 범죄 현황 분석

강남 3구의 체감 안전도가 높다는 기사를 검증하려 합니다. 

## 2-1 데이터 획득하기

구글에서 '서울시 관서별 5대 범죄 발생 검거 현황'으로 검색해도 되고, data.go.kr 에서 공공데이터 포럼으로 원하는 데이터를 다운받습니다.

## 2-2 pandas를 이요한 데이터 정리하기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

다운받은 데이터를 pandas로 읽어, `crime_anal_police`라는 변수에 저장합니다.
- 그 내용에는 서울시 경찰서별로 살인, 강도, 강간, 절도, 폭력 이라는 5대 범죄에 발생 건수와 검거 건수를 가지고 있습니다.

In [3]:
crime_anal_police = pd.read_csv('/content/drive/MyDrive/파이썬으로_데이터_주무르기/data/02. crime_in_Seoul.csv', thousands = ',', encoding = 'cp949')

crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,1395,477,1355,1170
1,종로서,3,3,6,5,115,98,1070,413,1278,1070
2,남대문서,1,0,6,4,65,46,1153,382,869,794
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711
4,혜화서,3,2,5,4,96,63,1114,424,1015,861


여기서 문제가 하나 발생합니다. 우리는 강남 3구가 안전한지를 확인해야 하는데, 데이터가 관서별로 되어있습니다. 서울시에는 한 구에 하나 혹은 두 군데의 경찰서가 위치해 있고, 구 이름과 다른 경찰서도 있습니다. 이 결찰서 목록을 소속 구별로 변경하고 싶습니다. 그러기 위해서는 먼저 경찰서 이름으로 구 정보를 알아야합니다. 양이 많지 않으니 직접해도 되지만, **<u>우리는 프로그램으로 접근하도록 하겠습니다.</u>**

## 2-3 지도 정보를 얻을 수 있는 Google Maps

Google Maps API는 구글에서 제공하는 서비스로, 위치에 대한 검색 결과 중 주소와 위도, 경도 정보를 제공합니다. 
- https://developers.google.com/maps/?hl=ko
- Google Maps API -> Google Maps Geocoding API(지오코딩) -> '키 가져오기'
- `pip install googlemaps`라는 명령어로 파이썬에서 googlemaps를 사용할 수 있도록 합니다.


In [4]:
pip install googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=b2033a51222c536cb21803e57e47805b1adc8cd0acc34a0f82dab59093694f59
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps


## 2-4 Google Maps를 이용해서 주소와 위도, 경도 정보 얻기

In [5]:
maps_key = pd.read_csv('/content/drive/MyDrive/파이썬으로_데이터_주무르기/data/GoogleAPIKey.txt')
maps_key = maps_key.columns[0] #공개되면 안되는 나의 GoogleMaps API key

In [6]:
import googlemaps

`googlemaps.Client`로 키를 입력합니다. key는, Google Maps API에서 제공받은 key를 사용합니다.

In [7]:
gmaps_key = maps_key
gmaps = googlemaps.Client(key = gmaps_key)

Google Maps를 사용해서 '서울중부경찰서' 라는 단어를 검색하면, `formatted_address`항목에 주소가 나오고, `lng`와 `lat`에서 위도 경도 정보 또한 확인할 수 있습니다.

In [8]:
gmaps.geocode('서울중부경찰서', language='ko')

[{'address_components': [{'long_name': '２７',
    'short_name': '２７',
    'types': ['premise']},
   {'long_name': '수표로',
    'short_name': '수표로',
    'types': ['political', 'sublocality', 'sublocality_level_4']},
   {'long_name': '중구',
    'short_name': '중구',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': '서울특별시',
    'short_name': '서울특별시',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': '대한민국',
    'short_name': 'KR',
    'types': ['country', 'political']},
   {'long_name': '100-032',
    'short_name': '100-032',
    'types': ['postal_code']}],
  'formatted_address': '대한민국 서울특별시 중구 수표로 27',
  'geometry': {'location': {'lat': 37.56361709999999, 'lng': 126.9896517},
   'location_type': 'ROOFTOP',
   'viewport': {'northeast': {'lat': 37.5649660802915,
     'lng': 126.9910006802915},
    'southwest': {'lat': 37.5622681197085, 'lng': 126.9883027197085}}},
  'partial_match': True,
  'place_id': 'ChIJc-9q5uSifDURLhQmr5wkXmc',
 

In [9]:
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('formatted_address'))
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('geometry'))
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('geometry')['location']['lat'])
print(gmaps.geocode('서울중부경찰서', language='ko')[0].get('geometry')['location']['lng'])

대한민국 서울특별시 중구 수표로 27
{'location': {'lat': 37.56361709999999, 'lng': 126.9896517}, 'location_type': 'ROOFTOP', 'viewport': {'northeast': {'lat': 37.5649660802915, 'lng': 126.9910006802915}, 'southwest': {'lat': 37.5622681197085, 'lng': 126.9883027197085}}}
37.56361709999999
126.9896517


하지만, 위의 `crime_anal_police`의 관서명을 보면, 중부서, 종로서 등으로 되어있습니다. 그러면 구글검색에서 주소가 제대로 나오지 않기 때문에, 서울**경찰서로 만들어야 겠습니다.

In [10]:
station_name = []

for name in crime_anal_police['관서명']:
    station_name.append('서울' + str(name[:-1])+ '경찰서')

station_name

['서울중부경찰서',
 '서울종로경찰서',
 '서울남대문경찰서',
 '서울서대문경찰서',
 '서울혜화경찰서',
 '서울용산경찰서',
 '서울성북경찰서',
 '서울동대문경찰서',
 '서울마포경찰서',
 '서울영등포경찰서',
 '서울성동경찰서',
 '서울동작경찰서',
 '서울광진경찰서',
 '서울서부경찰서',
 '서울강북경찰서',
 '서울금천경찰서',
 '서울중랑경찰서',
 '서울강남경찰서',
 '서울관악경찰서',
 '서울강서경찰서',
 '서울강동경찰서',
 '서울종암경찰서',
 '서울구로경찰서',
 '서울서초경찰서',
 '서울양천경찰서',
 '서울송파경찰서',
 '서울노원경찰서',
 '서울방배경찰서',
 '서울은평경찰서',
 '서울도봉경찰서',
 '서울수서경찰서']

다음은 위 코드에서 만들어진 이름을 이용하여 googlemaps에서 주소를 받아옵니다.

- '서울종암경찰서'에서 계속 에러가 나서, geocode에러를 찾아보아도 명확한 해결이 되지않아서, 우선 '서울종암경찰' 로 대체하는것으로 통과시켰다.

In [11]:
station_address = []
station_lat = []
station_lng = []

for name in station_name:
    try:
        tmp = gmaps.geocode(name, language = 'ko')
        station_address.append(tmp[0].get("formatted_address"))

        tmp_loc = tmp[0].get('geometry')

        station_lat.append(tmp_loc['location']['lat'])
        station_lng.append(tmp_loc['location']['lat'])

        print(name + ' --> ' + tmp[0].get("formatted_address"))
    except:
        print("error at ",name)
        tmp = gmaps.geocode(name[:-1], language = 'ko')
        station_address.append(tmp[0].get("formatted_address"))

        tmp_loc = tmp[0].get('geometry')

        station_lat.append(tmp_loc['location']['lat'])
        station_lng.append(tmp_loc['location']['lat'])

        print(name[:-1] + ' --> ' + tmp[0].get("formatted_address"))
        continue

서울중부경찰서 --> 대한민국 서울특별시 중구 수표로 27
서울종로경찰서 --> 대한민국 서울특별시 종로구 종로1.2.3.4가동 율곡로 46
서울남대문경찰서 --> 대한민국 서울특별시 중구 한강대로 410
서울서대문경찰서 --> 대한민국 서울특별시 서대문구 통일로 113
서울혜화경찰서 --> 대한민국 서울특별시 종로구 창경궁로 112-16
서울용산경찰서 --> 대한민국 서울특별시 용산구 백범로 329
서울성북경찰서 --> 대한민국 서울특별시 성북구 삼선동 보문로 170
서울동대문경찰서 --> 대한민국 서울특별시 동대문구 약령시로21길 29
서울마포경찰서 --> 대한민국 서울특별시 마포구 마포대로 183
서울영등포경찰서 --> 대한민국 서울특별시 영등포구 당산동3가 국회대로 608
서울성동경찰서 --> 대한민국 서울특별시 성동구 행당동 왕십리광장로 9
서울동작경찰서 --> 대한민국 서울특별시 동작구 노량진로 148
서울광진경찰서 --> 대한민국 서울특별시 광진구 구의동 자양로 167
서울서부경찰서 --> 대한민국 서울특별시 은평구 녹번동 177-15
서울강북경찰서 --> 대한민국 서울특별시 강북구 오패산로 406
서울금천경찰서 --> 대한민국 서울특별시 금천구 시흥대로73길 50
서울중랑경찰서 --> 대한민국 서울특별시 중랑구 묵2동 249-2
서울강남경찰서 --> 대한민국 서울특별시 강남구 테헤란로114길 11
서울관악경찰서 --> 대한민국 서울특별시 관악구 관악로5길 33
서울강서경찰서 --> 대한민국 서울특별시 강서구 화곡6동 980-27
서울강동경찰서 --> 대한민국 서울특별시 강동구 성내로 57
error at  서울종암경찰서
서울종암경찰 --> 대한민국 서울특별시 성북구 종암동
서울구로경찰서 --> 대한민국 서울특별시 구로구 가마산로 235
서울서초경찰서 --> 대한민국 서울특별시 서초구 서초3동 반포대로 179
서울양천경찰서 --> 대한민국 서울특별시 양천구 목동동로 99
서울송파경찰서 --> 대한민국 서울특별시 송파구 중대로 221
서울노원경찰서 

저장 된 전체 주소를 확인합니다.

In [12]:
station_address

['대한민국 서울특별시 중구 수표로 27',
 '대한민국 서울특별시 종로구 종로1.2.3.4가동 율곡로 46',
 '대한민국 서울특별시 중구 한강대로 410',
 '대한민국 서울특별시 서대문구 통일로 113',
 '대한민국 서울특별시 종로구 창경궁로 112-16',
 '대한민국 서울특별시 용산구 백범로 329',
 '대한민국 서울특별시 성북구 삼선동 보문로 170',
 '대한민국 서울특별시 동대문구 약령시로21길 29',
 '대한민국 서울특별시 마포구 마포대로 183',
 '대한민국 서울특별시 영등포구 당산동3가 국회대로 608',
 '대한민국 서울특별시 성동구 행당동 왕십리광장로 9',
 '대한민국 서울특별시 동작구 노량진로 148',
 '대한민국 서울특별시 광진구 구의동 자양로 167',
 '대한민국 서울특별시 은평구 녹번동 177-15',
 '대한민국 서울특별시 강북구 오패산로 406',
 '대한민국 서울특별시 금천구 시흥대로73길 50',
 '대한민국 서울특별시 중랑구 묵2동 249-2',
 '대한민국 서울특별시 강남구 테헤란로114길 11',
 '대한민국 서울특별시 관악구 관악로5길 33',
 '대한민국 서울특별시 강서구 화곡6동 980-27',
 '대한민국 서울특별시 강동구 성내로 57',
 '대한민국 서울특별시 성북구 종암동',
 '대한민국 서울특별시 구로구 가마산로 235',
 '대한민국 서울특별시 서초구 서초3동 반포대로 179',
 '대한민국 서울특별시 양천구 목동동로 99',
 '대한민국 서울특별시 송파구 중대로 221',
 '대한민국 서울특별시 노원구 하계동 노원로 283',
 '대한민국 서울특별시 서초구 동작대로 204',
 '대한민국 서울특별시 은평구 불광동 연서로 365',
 '대한민국 서울특별시 도봉구 노해로 403',
 '대한민국 서울특별시 강남구 개포로 617']

In [13]:
station_lat

[37.56361709999999,
 37.575548,
 37.5547584,
 37.5647439,
 37.5719679,
 37.5387099,
 37.58977830000001,
 37.58506149999999,
 37.550814,
 37.5260441,
 37.5617303,
 37.5130866,
 37.542873,
 37.6021405,
 37.63730390000001,
 37.4568722,
 37.6056429,
 37.5094352,
 37.4743945,
 37.551284,
 37.528511,
 37.60017089999999,
 37.494931,
 37.4956054,
 37.5167711,
 37.5016941,
 37.6425238,
 37.4945959,
 37.6283178,
 37.6533589,
 37.49349]

In [14]:
station_lng

[37.56361709999999,
 37.575548,
 37.5547584,
 37.5647439,
 37.5719679,
 37.5387099,
 37.58977830000001,
 37.58506149999999,
 37.550814,
 37.5260441,
 37.5617303,
 37.5130866,
 37.542873,
 37.6021405,
 37.63730390000001,
 37.4568722,
 37.6056429,
 37.5094352,
 37.4743945,
 37.551284,
 37.528511,
 37.60017089999999,
 37.494931,
 37.4956054,
 37.5167711,
 37.5016941,
 37.6425238,
 37.4945959,
 37.6283178,
 37.6533589,
 37.49349]

이제, 저장한 주소를, 띄어쓰기, 공백으로 나누고(split) 두 번쨰 단어(얼핏보니 구이름)를 선택해서 '구별'이라는 컬럼으로 저장합니다. 이렇게하면 관서명에서 google maps의 도움을 받아 구별 이름으로 저장할 수 있게 되었습니다.

In [15]:
gu_name = []
for name in station_address:
    tmp = name.split()

    tmp_gu = [gu for gu in tmp if gu[-1] == '구'][0]

    gu_name.append(tmp_gu)

crime_anal_police['구별'] = gu_name
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구


~~단, 금천경찰서의 경우에는 관악구에 위치 해 있어서, 금천서는 예외처리를 해야합니다.~~ 책에서는 그렇게 하였으나, 내겐 이미 업데이트 되어있어서 통과.

In [16]:
crime_anal_police[crime_anal_police['관서명']=='금천서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
15,금천서,3,4,6,6,151,122,1567,888,2054,1776,금천구


~~관악구로 되어있는것을 금천구로 변경~~

In [17]:
# crime_anal_police.loc[crime_anal_police['관서명']=='금천서', ['구별']] = '금천서'
# crime_anal_police[crime_anal_police['관서명']=='금천서']

인터넷에서 자료를 가져오는 경우는 이 과정을 단순한 프로그램 오류로 다시 수행할 때 발생하는 정신적 스트레스를 피하기 위해서 적당한 이름으로 저장합니다. 이 때 한 가지 고민이 생깁니다.

In [18]:
crime_anal_police.to_csv('/content/drive/MyDrive/파이썬으로_데이터_주무르기/data/02. crime_in_Seoul_include_gu_name.csv',
                         sep = ',', encoding = 'utf-8')

현재까지 우리가 확보한 데이터의 구조가 관서명을 기초로 했기 때문에 구별 컬럼에서는 같은 구 이름이 두 번 있을 수 있습니다. 우리는 이 부분을 어떻게 해야할 지 공부해야 합니다.

In [19]:
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,1395,477,1355,1170,중구
1,종로서,3,3,6,5,115,98,1070,413,1278,1070,종로구
2,남대문서,1,0,6,4,65,46,1153,382,869,794,중구
3,서대문서,2,2,5,4,154,124,1812,738,2056,1711,서대문구
4,혜화서,3,2,5,4,96,63,1114,424,1015,861,종로구


## 2-5 pandas의 `pivot_table` 학습하기

pandas 관련 많은 예제 중 github를 운영 중이신 chris1610님이 공개한 데이터와 예제를 인용했다고 합니다.
> https://github.com/chris1610/pbpython/tree/master/data

In [22]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/drive/MyDrive/파이썬으로_데이터_주무르기/data/02. sales-funnel.xlsx')
df.head()

,Account,Name,Rep,Manager,Product,Quantity,Price,Status
0,714466,Trantow-Barrows,Craig Booker,Debra Henley,CPU,1,30000,presented
1,714466,Trantow-Barrows,Craig Booker,Debra Henley,Software,1,10000,presented
2,714466,Trantow-Barrows,Craig Booker,Debra Henley,Maintenance,2,5000,pending
3,737550,"Fritsch, Russel and Anderson",Craig Booker,Debra Henley,CPU,1,35000,declined
4,146832,Kiehn-Spinka,Daniel Hilton,Debra Henley,CPU,2,65000,won


위 데이터에서 Name 항목으로만 정렬할 때 pivot_table을 사용합니다.
- 그러면 Name 칼럼이 index가 되고 특별히 지정하지 않았다면 숫자형 데이터 컬럼들이 남게 됩니다. 
- 그리고 중복 된 Name의 항목은 하나로 합쳐지고 value들은 평균을 갖게 됩니다.

In [23]:
pd.pivot_table(df,index=['Name'])

,Account,Price,Quantity
Name,,,
Barton LLC,740150,35000,1.000000
"Fritsch, Russel and Anderson",737550,35000,1.000000
Herman LLC,141962,65000,2.000000
Jerde-Hilpert,412290,5000,2.000000
"Kassulke, Ondricka and Metz",307599,7000,3.000000
Keeling LLC,688981,100000,5.000000
Kiehn-Spinka,146832,65000,2.000000
Koepp Ltd,729833,35000,2.000000
Kulas Inc,218895,25000,1.500000
